In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [3]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(37)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}
ensemble_weights = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.77275561 0.78177431 0.78687489 0.78537141 0.78830814 0.77508712
 0.78963673 0.79400626 0.79616168 0.79517875 0.78959082 0.80151477
 0.80186191 0.80079904 0.80238704 0.78698985 0.7936223  0.79393447
 0.79971561 0.80272708 0.78394136 0.7884767  0.79113748 0.79303668
 0.80047043        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [31 17], Predictions: [0 1], [39  9]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [31 17], Predictions: [0 1], [34 14]
Training RF ...
Model RF, Actual: [0 1], [31 17], Predictions: [0 1], [39  9]
Training XGB ...
Model XGB, Actual: [0 1], [31 17], Predictions: [0 1], [36 12]

None


[('lf_hf_ratio', 87), ('sdnn', 78), ('eda_std', 72), ('rmssd', 69), ('ecg_var', 66), ('mean_SCL', 60), ('SCR_rate', 58), ('eda_median', 58), ('hf_rr', 57), ('bpm', 53), ('ec

One or more of the test scores are non-finite: [0.79933614 0.81173166 0.82410884 0.82651755 0.8255367  0.79270998
 0.80890007 0.81546638 0.81905933 0.82178598 0.80640586 0.81719688
 0.82047321 0.81898406 0.8171188  0.81406581 0.81947377 0.83403047
 0.8322484  0.83012956 0.80450103 0.82006124 0.82764934 0.82547654
 0.82954683        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [33 15], Predictions: [0 1], [47  1]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [33 15], Predictions: [0 1], [38 10]
Training RF ...
Model RF, Actual: [0 1], [33 15], Predictions: [0 1], [39  9]
Training XGB ...
Model XGB, Actual: [0 1], [33 15], Predictions: [0 1], [39  9]

None


[('lf_hf_ratio', 123), ('sdnn', 112), ('ecg_var', 104), ('rmssd', 102), ('mean_SCL', 99), ('SCR_rate', 97), ('eda_std', 87), ('ecg_mean', 86), ('eda_median', 80), ('eda_var', 76), (

...

Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [42  6], Predictions: [0], [48]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [42  6], Predictions: [0 1], [45  3]
Training RF ...
Model RF, Actual: [0 1], [42  6], Predictions: [0 1], [46  2]
Training XGB ...
Model XGB, Actual: [0 1], [42  6], Predictions: [0 1], [47  1]

None


[('sdnn', 114), ('lf_hf_ratio', 104), ('eda_std', 96), ('mean_SCL', 93), ('bpm', 85), ('eda_mean', 84), ('rmssd', 81), ('ecg_var', 81), ('SCR_rate', 77), ('eda_median', 75), ('ecg_std

One or more of the test scores are non-finite: [0.78987189 0.8068677  0.81540751 0.82260713 0.82222257 0.78878983
 0.80760786 0.80914439 0.81804746 0.81827886 0.79368568 0.79977546
 0.81560136 0.81535003 0.81165792 0.80373798 0.81756927 0.82119439
 0.81937876 0.82113613 0.81273594 0.81394081 0.81630448 0.81442444
 0.81448442        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [44  4], Predictions: [0 1], [45  3]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [44  4], Predictions: [0 1], [43  5]
Training RF ...
Model RF, Actual: [0 1], [44  4], Predictions: [0 1], [43  5]
Training XGB ...
Model XGB, Actual: [0 1], [44  4], Predictions: [0 1], [42  6]

None


[('sdnn', 61), ('mean_SCL', 61), ('lf_hf_ratio', 61), ('eda_std', 52), ('rmssd', 48), ('eda_median', 43), ('bpm', 42), ('ecg_var', 40), ('ecg_std', 39), ('hf_rr', 36), ('eda_m

One or more of the test scores are non-finite: [0.7916409  0.80740708 0.80462881 0.80480203 0.8110083  0.77990659
 0.79013372 0.80308569 0.8056199  0.80554789 0.78366222 0.78263487
 0.78836879 0.79646182 0.79797159 0.78069971 0.78338842 0.78579755
 0.79110397 0.78983503 0.75989692 0.78614956 0.79193978 0.79800199
 0.79598025        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [38 10], Predictions: [0 1], [40  8]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [38 10], Predictions: [0 1], [38 10]
Training RF ...
Model RF, Actual: [0 1], [38 10], Predictions: [0 1], [39  9]
Training XGB ...
Model XGB, Actual: [0 1], [38 10], Predictions: [0 1], [40  8]

None


[('rmssd', 45), ('sdnn', 43), ('eda_median', 40), ('mean_SCL', 37), ('lf_hf_ratio', 37), ('SCR_rate', 34), ('ecg_median', 33), ('eda_std', 32), ('eda_mean', 30), ('ecg_mean'

In [4]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(37)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=45)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
weights = [ensemble_weights[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
# weights = np.divide(weights, len(weights))
weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.8375
	Precision: 0.8181818181818182
	Recall: 0.45
	F1-score: 0.5806451612903226
	AUC score: 0.7083333333333333
----------------------------------------
